In [2]:
# Imports

import pandas as pd

path = 'insert_path'

games = pd.read_excel(f"{path}CBB Conference Comparison Sheet.xlsx", sheet_name = 'Games')
games = games.iloc[:, 0:9]

conferences = pd.read_excel(f"{path}CBB Conference Comparison Sheet.xlsx", sheet_name = 'Conferences')

In [3]:
def game_conference_merge(games = games, 
                          conferences = conferences, 
                          season = 'all', 
                          power_6 = False, 
                          games_column = 'Tm', 
                          conferences_column = 'School'):
    
    """
    Function: 
        Merge the Games Sheet with the Conference Sheet, 
        pairing all teams with their respective conferences.
        
        Note: This function does not remove games of teams not in studied conferences.
    
    Variables:
        games: Spreadsheet of games
        conferences: Spreadsheet of conferences
        season: Which season to pull data from (2021,2022, or 2023). 'all' combines all 3 years
        power_6: Only choose Power 6 Conferences (exclude MWC, AAC, and WCC)
        games_column: Team name column in games spreadsheet ('Tm')
        conferences_column: Team name column in conferences spreadsheet ('School')
    """
    
    if power_6 == True:
        conference_list = ['ACC', 'Big 12', 'Big East', 'Big Ten', 'Pac-12', 'SEC']
    else:
        conference_list = ['AAC', 'ACC', 'Big 12', 'Big East', 'Big Ten', 'MWC', 'Pac-12', 'SEC', 'WCC']
    
    if season == 'all':
        season_list = [2021,2022,2023]
    else:
        season_list = [season]
    
    return_df = pd.DataFrame() # finish this later oh my god did I fuck this?
    
    for season in season_list:
        games_prep = games.loc[games['Season'] == season, :]
        conferences_prep = conferences.loc[(conferences['Season'] == season) & (conferences['Conf'].isin(conference_list)), :]

        merge_df = (
        pd.merge(games_prep,conferences_prep,
        how="left",
        left_on= games_column,
        right_on= conferences_column)
        .rename(columns = {'Season_x':'Season',
                           'Unnamed: 3':'Site',
                           'Conf':f'{games_column}_Conf',
                           'W':f'{games_column}_W',
                           'L':f'{games_column}_L',
                           '%':f'{games_column}_%'})
        )
        
        return_df = pd.concat([return_df, merge_df], axis=0)
        
    return(return_df)

def expected_metrics(games = games, 
                     conferences = conferences, 
                     season = 'all',
                     power_6 = False):
    
    """
    Function: 
        Runs conference merge function and computes expected point differentials, 
        implied outcomes, and alphas.
        
        Note: This function does remove games of teams not in studied conferences.
    
    Variables:
        games: Spreadsheet of games
        conferences: Spreadsheet of conferences
        season: Which season to pull data from (2021,2022, or 2023). 'all' combines all 3 years
        power_6: Only choose Power 6 Conferences (exclude MWC, AAC, and WCC)
    """
    
    Tm_merge = game_conference_merge(games, conferences, season, power_6 = power_6)
    Opp_merge = game_conference_merge(Tm_merge, conferences, season, power_6 = power_6, games_column = 'Opp')

    conf_df = (Opp_merge
         .loc[Opp_merge['Tm_Conf']== Opp_merge['Opp_Conf'],:]
         .dropna(subset = ['Opp_Conf'])
        )
    
    diffs = conf_df.groupby(by = ['Tm','Season'])['Diff'].sum()
        
    nonconf_df = (Opp_merge
             .loc[Opp_merge['Tm_Conf']!= Opp_merge['Opp_Conf'],:]
             .dropna(subset = ['Tm_Conf','Opp_Conf'])
             .reset_index(drop = True)
            )
    
    Tm_diff_df = (pd.merge(nonconf_df, diffs, how='left', left_on=['Season','Tm'], right_on = ['Season', 'Tm'])
    .rename(columns = {'Diff_x':'Game_Diff',
                       'Diff_y':'Tm_Diff'}) 
                 )
    
    return_df = (pd.merge(Tm_diff_df, diffs, how='left', left_on=['Season','Opp'], right_on = ['Season','Tm'])
    .rename(columns = {'Diff':'Opp_Diff'}) 
    .assign(Imp_Prob=lambda df: df['Tm_%']/(df['Tm_%'] + df['Opp_%']),
            Tm_Diff_Avg = lambda df: df['Tm_Diff']/(df['Tm_W'] + df['Tm_L']),
            Opp_Diff_Avg = lambda df: df['Opp_Diff']/(df['Opp_W'] + df['Opp_L']),
            Exp_Diff = lambda df: df['Tm_Diff_Avg']-df['Opp_Diff_Avg'],
            Outcome = lambda df: df['Game_Diff'].apply(lambda x: 1 if x > 0 else 0),
            Outcome_Alpha = lambda df: df['Outcome'] - df['Imp_Prob'],
            Diff_Alpha = lambda df: df['Game_Diff'] - df['Exp_Diff'])
            )


    column_order = ['Season','Date','Tm','Tm_Conf','Tm_W','Tm_L','Tm_%','Tm_Diff','Tm_Diff_Avg','Site','Opp','Opp_Conf', 'Opp_W', 'Opp_L', 'Opp_%','Opp_Diff','Opp_Diff_Avg','W/L','Tm_Score','Opp_Score','Outcome','Imp_Prob','Outcome_Alpha','Game_Diff','Exp_Diff','Diff_Alpha']
    
    return_df = return_df[column_order]
    
    return(return_df)

def conference_data(df, 
                    cross_conference = True):
    
    """
    Function: 
        Takes in expected_metrics output and creates conferences summary statistics 
        and alphas for implied outcomes and point differentials
    
    Variables:
        df: output of expected_metrics()
        cross_conferences: Boolean, whether or to also group by opponent conference
    """
    
    if cross_conference == True:
        groupby_list = ['Tm_Conf','Opp_Conf']
        column_order = ['Season', 'Conference', 'Opponent_Conference', 'Games', 'Wins', 'Losses', 'Win_Percentage','Expected_Wins',
       'Expected_Losses','Expected_Win_Percentage','Win_Percentage_Alpha', 'Point_Differential_Alpha',
       'Point_Differential_Alpha_STD']
    else:
        groupby_list = ['Tm_Conf']
        column_order = ['Season','Conference', 'Games', 'Wins', 'Losses', 'Win_Percentage','Expected_Wins',
       'Expected_Losses','Expected_Win_Percentage','Win_Percentage_Alpha', 'Point_Differential_Alpha',
       'Point_Differential_Alpha_STD']
        
    summary_df = (df
     .groupby(groupby_list)
     .agg({'Season':['mean'],'Outcome': ['count','sum'],'Imp_Prob':['sum'],'Outcome_Alpha': ['mean'], 'Diff_Alpha': ['mean','std']})
    )
    
    summary_df.columns = [f'{col[0]}_{col[1]}' for col in summary_df.columns]
    summary_df = (summary_df
                  .reset_index()
                  .rename(columns = {
                      'Season_mean':'Season',
                      'Tm_Conf':'Conference',
                      'Opp_Conf':'Opponent_Conference',
                      'Outcome_count':'Games',
                      'Outcome_sum':'Wins',
                      'Imp_Prob_sum':'Expected_Wins',
                      'Outcome_Alpha_mean' :'Win_Percentage_Alpha',
                      'Diff_Alpha_mean': 'Point_Differential_Alpha',
                      'Diff_Alpha_std':'Point_Differential_Alpha_STD'

                  })
                  .assign(Losses = lambda df: df['Games'] - df['Wins'],
                         Expected_Losses = lambda df: df['Games']-df['Expected_Wins'],
                         Win_Percentage = lambda df: df['Wins']/df['Games'],
                         Expected_Win_Percentage = lambda df: df['Expected_Wins']/df['Games'])
                 )

    
    return_df = summary_df[column_order]
    
    return(return_df)
    

def final_sheet(games, 
                conferences, 
                season = 'all', 
                cross_conference = True, 
                export = False, 
                power_6 = False):
    
    """
    Function: 
        All encompassing function, convert games and conferences spreadsheets to
        summary statistics ready for visualization
    
    Variables:
        games: Spreadsheet of games
        conferences: Spreadsheet of conferences
        season: Which season to pull data from (2021,2022, or 2023). 'all' combines all 3 years
        cross_conferences: Boolean, whether or to also group by opponent conference
        export: Boolean, whether to export final spreadsheet
        power_6: Only choose Power 6 Conferences (exclude MWC, AAC, and WCC)
    """
    
    df = expected_metrics(games, conferences, season, power_6 = power_6)
    return_df = conference_data(df, cross_conference)
    
    if cross_conference == True:
        export_name = f'{season}_CBB_Cross_Conference_Comparison.csv'
    else:
        export_name = f'{season}_CBB_Conference_Comparison.csv'
    
    if export == True:
        return_df.to_csv(export_name)
        
    return(return_df)

def exports(power_6 = True):
    
    """
    Function: 
        Run final_sheet function for all combinations of seasons and cross conferences 
        compiled into a single spreadsheet
    
    Variables:
        power_6: Only choose Power 6 Conferences (exclude MWC, AAC, and WCC)
    """ 
    season_list = [2021,2022,2023]

    return_df = pd.DataFrame()
    return_df_cross = pd.DataFrame()

    for season in season_list:

        no_cross = final_sheet(games = games,                   
                           conferences = conferences, 
                           season = season, 
                           cross_conference = False,
                           export = False,
                           power_6 = power_6)

        cross = final_sheet(games = games, 
                        conferences = conferences, 
                        season = season, 
                        cross_conference = True,
                        export = False,
                        power_6 = power_6)

        return_df = pd.concat([return_df, no_cross], axis=0)
        return_df_cross = pd.concat([return_df_cross, cross], axis=0)
        
    all_df = final_sheet(games = games, 
                    conferences = conferences, 
                    season = 'all', 
                    cross_conference = False,
                    export = False,
                    power_6 = power_6)

    all_df_cross = final_sheet(games = games, 
                    conferences = conferences, 
                    season = 'all', 
                    cross_conference = True,
                    export = False,
                    power_6 = power_6)
    
    return_df.to_csv('CBB_Conference.csv')
    return_df_cross.to_csv('CBB_Cross.csv')
    all_df.to_csv('CBB_Conference_All_Season.csv')
    all_df_cross.to_csv('CBB_Cross_All_Season.csv')



In [ ]:
exports(power_6 = True)